# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
On drop dispersion and mixing performance in a stirred vessel: effect of surfactant solubility	Conference presentation https://meetings.aps.org/Meeting/DFD23/Session/T18.5        2023-11-20  Washington D.C. USA     
Tutorial 1 on Relevant Topic in Your Field	Tutorial	tutorial-1	UC-Berkeley Institute for Testing Science	2013-03-01	Berkeley CA, USA	http://exampleurl.com	This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
Talk 2 on Relevant Topic in Your Field	Talk	talk-2	London School of Testing	2014-02-01	London, UK	http://example2.com	This is a description of your talk, which is a markdown files that can be all markdown-ified like any other post. Yay markdown!
Conference Proceeding talk 3 on Relevant Topic in Your Field	Conference proceedings talk	talk-3	Testing Institute of America 2014 Annual Conference	2014-03-0

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [12]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,On drop dispersion and mixing performance in a...,Conference presentation,talk-1,NaN,2023-11-20,"Washington D.C., USA",NaN,76th Annual Meeting of the APS Division of Flu...
1,Machine Learning for Prediction and Optimizati...,Conference presentation,talk-2,NaN,2023-11-07,"Orlando, USA",NaN,2023 AIChE annual meeting
2,Numerical simulation of surfactant-laden emuls...,Conference presentation,talk-3,NaN,2023-04-07,"Kobe, Japan",NaN,11th International Conference on Multiphase Flow
3,Numerical simulation of surfactant-laden emuls...,Conference presentation,talk-4,NaN,2022-11-21,"Indianapolis, USA",NaN,75th Annual Meeting of the APS Division of Flu...
4,Numerical simulation of surfactant-laden emuls...,Conference presentation,talk-5,NaN,2022-11-15,"Pheonix, USA",NaN,2022 AIChE annual meeting
5,A numerical vinaigrette: A myriad of interfaci...,Conference presentation,talk-6,NaN,2022-09-13,"Athens, Greece",NaN,14th European Fluid Mechanics Conference
6,A numerical vinaigrette: A myriad of interfaci...,Conference presentation,talk-7,NaN,2022-06-21,Online,NaN,"Mixing XXVII, the North American Mixing Forum"
7,A Numerical Vinaigrette: Effect of Surfactants...,Conference presentation,talk-8,NaN,2022-06-09,"Oslo, Norway",NaN,8th European Congress on Computational Methods...
8,Effect of Impeller Rotation on Oil-water Emuls...,Conference presentation,talk-9,NaN,2021-11-20,Online,NaN,74th Annual Meeting of the APS Division of Flu...
9,Tutorial 1 on Relevant Topic in Your Field,Tutorial,tutorial-1,UC-Berkeley Institute for Testing Science,2013-03-01,"Berkeley CA, USA",http://exampleurl.com,"This is a description of your tutorial, note t..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [13]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [15]:
!ls ../_talks

2012-03-01-talk-1.md     2022-06-09-talk-8.md     2023-04-07-talk-3.md
2013-03-01-tutorial-1.md 2022-06-21-talk-7.md     2023-11-07-talk-2.md
2014-02-01-talk-2.md     2022-09-13-talk-6.md     2023-11-20-talk-1.md
2014-03-01-talk-3.md     2022-11-15-talk-5.md
2021-11-20-talk-9.md     2022-11-21-talk-4.md


In [16]:
# !cat ../_talks/2013-03-01-tutorial-1.md
!cat ../_talks/2022-06-09-talk-8.md

---
title: "A Numerical Vinaigrette: Effect of Surfactants on Oil-water Emulsification"
collection: talks
type: "Conference presentation"
permalink: /talks/2022-06-09-talk-8
date: 2022-06-09
location: "Oslo, Norway"
---

8th European Congress on Computational Methods in Applied Sciences and Engineering
